# Clase Práctica #15 (Compilación)

En esta clase implementaremos la fase final de chequeo semántico para el lenguaje que comenzamos a estudiar en clases anteriores. Pasemos a importar lo que ya habíamos implementado.

### Análisis Lexicográfico y Sintáctico

In [11]:
import cmp.nbpackage
import cmp.visitor as visitor

from cp13 import G, text
from cp13 import Node, ProgramNode, DeclarationNode, ExpressionNode
from cp13 import ClassDeclarationNode, FuncDeclarationNode, AttrDeclarationNode
from cp13 import VarDeclarationNode, AssignNode, CallNode
from cp13 import AtomicNode, BinaryNode
from cp13 import ConstantNumNode, VariableNode, InstantiateNode, PlusNode, MinusNode, StarNode, DivNode
from cp13 import FormatVisitor, tokenize_text, pprint_tokens

from cmp.tools.parsing import LR1Parser
from cmp.evaluation import evaluate_reverse_parse

### Análisis Semántico (Recolección y Construcción de Tipos)

In [12]:
from cmp.semantic import SemanticError
from cmp.semantic import Attribute, Method, Type
from cmp.semantic import VoidType, ErrorType, IntType
from cmp.semantic import Context

from cp14 import TypeCollector, TypeBuilder, run_pipeline

## Chequeo de tipos

Estaremos validando que el programa haga un uso correcto de los tipos definidos. Recordemos algunas de las características del lenguaje que queremos comprobar:
- Todos los métodos son de instancia y dentro de ellos es visible `self` _(solo lectura)_, cuyo tipo estático coincide con el de la clase que implementa el método.
- Al invocar un método, se evalúa primero la expresión que devuelve el objeto, luego los parámetros, y por último se llama la función.
- Todos los atributos son privados y todos los métodos son públicos.
- Un método se puede sobrescribir sí y solo sí se mantiene exactamente la misma definición para los tipos de retorno y de los argumentos.
- La expresión `let` tiene la sintaxis: `let <var>: <type> = <expr>` y evalúa al valor de la expresión (y con el tipo estático de la expresión). La variable `<var>` no puede estar previamente definida en ese ámbito. Se puede omitir la declaración del tipo y en tal caso la variable `<var>` ya debe estar definida y se realiza la asignación.
- Las operaciones `+`, `-`, `*` y `/` están definidas únicamente entre valores enteros, y devuelven enteros.

In [13]:
WRONG_SIGNATURE = 'Method "%s" already defined in "%s" with a different signature.'
SELF_IS_READONLY = 'Variable "self" is read-only.'
LOCAL_ALREADY_DEFINED = 'Variable "%s" is already defined in method "%s".'
INCOMPATIBLE_TYPES = 'Cannot convert "%s" into "%s".'
VARIABLE_NOT_DEFINED = 'Variable "%s" is not defined in "%s".'
INVALID_OPERATION = 'Operation is not defined between "%s" and "%s".'

La clase `Type` ha sido extendida para incluir la función `conforms_to`. Recordemos que esta relación indica que un objeto de tipo `C` puede ser usado en lugar de un objeto de tipo `P`. En tal caso decimos que `C.conforms_to(P)`. La relación de conformidad es crucial en los lenguajes de programación orientados a objetos pues restringirá las asignaciones e invocaciones para garantizar el principio de sustitución.

### Ámbito (Scope)

Se provee una implementación de la clase `Scope` en `cmp.semantic`. Esta clase nos permitirá gestionar las variables definidas en los distintos niveles de visibilidad, así como saber con qué tipo se definieron. Los métodos fundamentales son:
- `create_child`: Crea un `scope` hijo que hereda las variables visibles hasta ese momento en el `scope` padre.
- `define_variable`: Registra localmente una variable en el `scope` a partir de su nombre y tipo.
- `find_variable`: Devuelve un `VariableInfo` con la información de la variable consultada (a partir de su nombre). La variable devuelta no tiene por qué estar definida localmente. En caso de que la variable no esté definida devuelve `None`.
- `is_defined`: Indica si la variable consultada es visible en el `scope`.
- `is_local`: Indica si la variable consultada está definida localmente en el `scope`.

In [14]:
from cmp.semantic import Scope

### Type Checker Visitor

Implementaremos un nuevo recorrido sobre el AST. Como de costumbre nos apoyaremos en el patrón visitor. Debemos caminar desde la raíz del AST (`ProgramNode`) hasta el cuerpo de los métodos. En esta ocasión además, verificaremos que los métodos construidos en el recorrido anterior no hayan tenido sobrecargas distintas a lo largo del árbol de herencia.

In [15]:
class TypeChecker:
    def __init__(self, context, errors=[]):
        self.context = context
        self.current_type = None
        self.current_method = None
        self.errors = errors

    @visitor.on('node')
    def visit(self, node, scope):
        pass

    @visitor.when(ProgramNode)
    def visit(self, node, scope=None):
        scope = Scope()
        for declaration in node.declarations:
            self.visit(declaration, scope.create_child())
        return scope

    @visitor.when(ClassDeclarationNode)
    def visit(self, node, scope):
        self.current_type = self.context.get_type(node.id)
        
        scope.define_variable('self', self.current_type)
        for attr in self.current_type.attributes:
            scope.define_variable(attr.name, attr.type)
            
        for feature in node.features:
            self.visit(feature, scope.create_child())
        
    @visitor.when(AttrDeclarationNode)
    def visit(self, node, scope):
        pass

    @visitor.when(FuncDeclarationNode)
    def visit(self, node, scope):
        self.current_method = self.current_type.get_method(node.id)
        
        for pname, ptype in zip(self.current_method.param_names, self.current_method.param_types):
            scope.define_variable(pname, ptype)
            
        for expr in node.body:
            self.visit(expr, scope)
            
        last_expr = node.body[-1]
        last_expr_type = last_expr.computed_type
        method_rtn_type = self.current_method.return_type
        
        if not last_expr_type.conforms_to(method_rtn_type):
            self.errors.append(INCOMPATIBLE_TYPES.replace('%s', last_expr_type.name, 1).replace('%s', method_rtn_type.name, 1))

    @visitor.when(VarDeclarationNode)
    def visit(self, node, scope):
        self.visit(node.expr, scope)
        expr_type = node.expr.computed_type
        
        try:
            node_type = self.context.get_type(node.type)
        except SemanticError as ex:
            self.errors.append(ex.text)
            node_type = ErrorType()
        
        if not expr_type.conforms_to(node_type):
            self.errors.append(INCOMPATIBLE_TYPES.replace('%s', expr_type.name, 1).replace('%s', node_type.name, 1))
          
        if not scope.is_defined(node.id):
            scope.define_variable(node.id, node_type)
        else:
            self.errors.append(LOCAL_ALREADY_DEFINED.replace('%s', node.id, 1).replace('%s', self.current_method.name, 1))
        
        node.computed_type = node_type
            
    @visitor.when(AssignNode)
    def visit(self, node, scope):
        self.visit(node.expr, scope)
        expr_type = node.expr.computed_type
        
        if scope.is_defined(node.id):
            var = scope.find_variable(node.id)
            node_type = var.type       
            
            if var.name == 'self':
                self.errors.append(SELF_IS_READONLY)
            elif not expr_type.conforms_to(node_type):
                self.errors.append(INCOMPATIBLE_TYPES.replace('%s', expr_type.name, 1).replace('%s', node_type.name, 1))
        else:
            self.errors.append(VARIABLE_NOT_DEFINED.replace('%s', node.id, 1).replace('%s', self.current_method.name, 1))
            node_type = ErrorType()
        
        node.computed_type = node_type
    
    @visitor.when(CallNode)
    def visit(self, node, scope):
        self.visit(node.obj, scope)
        obj_type = node.obj.computed_type
        
        try:
            obj_method = obj_type.get_method(node.id)
            
            if len(node.args) == len(obj_method.param_types):
                for arg, param_type in zip(node.args, obj_method.param_types):
                    self.visit(arg, scope)
                    arg_type = arg.computed_type
                    
                    if not arg_type.conforms_to(param_type):
                        self.errors.append(INCOMPATIBLE_TYPES.replace('%s', arg_type.name, 1).replace('%s', param_type.name, 1))
            else:
                self.errors.append(f'Method "{obj_method.name}" of "{obj_type.name}" only accepts {len(obj_method.param_types)} argument(s)')
            
            node_type = obj_method.return_type
        except SemanticError as ex:
            self.errors.append(ex.text)
            node_type = ErrorType()
            
        node.computed_type = node_type
    
    @visitor.when(BinaryNode)
    def visit(self, node, scope):
        self.visit(node.left, scope)
        left_type = node.left.computed_type
        
        self.visit(node.right, scope)
        right_type = node.right.computed_type
        
        if not left_type.conforms_to(IntType()) or not right_type.conforms_to(IntType()):
            self.errors.append(INVALID_OPERATION.replace('%s', left_type.name, 1).replace('%s', right_type.name, 1))
            node_type = ErrorType()
        else:
            node_type = IntType()
            
        node.computed_type = node_type
    
    @visitor.when(ConstantNumNode)
    def visit(self, node, scope):
        node.computed_type = IntType()

    @visitor.when(VariableNode)
    def visit(self, node, scope):
        if scope.is_defined(node.lex):
            var = scope.find_variable(node.lex)
            node_type = var.type       
        else:
            self.errors.append(VARIABLE_NOT_DEFINED.replace('%s', node.lex, 1).replace('%s', self.current_method.name, 1))
            node_type = ErrorType()
        
        node.computed_type = node_type

    @visitor.when(InstantiateNode)
    def visit(self, node, scope):
        try:
            node_type = self.context.get_type(node.lex)
        except SemanticError as ex:
            self.errors.append(ex.text)
            node_type = ErrorType()
            
        node.computed_type = node_type


### Pipeline

Actualicemos el método `run_pipeline` para incluir esta nueva fase. Con eso deberíamos completar una línea de ejecución para llegar al final del chequeo semántico partiendo desde el programa en texto plano y la gramática.

In [16]:
from cp14 import run_pipeline as deprecated_pipeline

def run_pipeline(G, text):
    ast, errors, context = deprecated_pipeline(G, text)
    print('=============== CHECKING TYPES ================')
    checker = TypeChecker(context, errors)
    scope = checker.visit(ast)
    print('Errors: [')
    for error in errors:
        print('\t', error)
    print(']')
    return ast, errors, context, scope

### Programa #1

El siguiente programa no debería contener errores.

In [17]:
text = '''
class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b ;
    }
    b : int ;
}

class B : A {
    c : int ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        c ;
    }
}
'''

if __name__ == '__main__':
    ast, errors, context, scope = run_pipeline(G, text)
    assert not errors

=================== TEXT ======================

class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b ;
    }
    b : int ;
}

class B : A {
    c : int ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        c ;
    }
}

================== TOKENS =====================
class id {
    id : id ;
    def id ( id : id , id : id ) : id {
        id + id ;
    }
    id : id ;
}
class id : id {
    id : id ;
    def id ( id : id , id : id ) : id {
        let id : id = int ;
        let id = new id ( ) . id ( int , id ) ;
        id ;
    }
}
$
=================== PARSE =====================
<def-attr> -> id : id ;
<param> -> id : id
<param> -> id : id
<param-list> -> <param>
<param-list> -> <param> , <param-list>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <term>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <arith> + <term>
<expr> -> <arith>
<expr-list> -> <expr> ;
<

### Programa #2

Se incluyeron varios errores al programa anterior. Intente detectar todos los errores posibles.

In [18]:
text = '''
class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b + new B ( ) ;
    }
    b : int ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        d ;        
    }
}
'''

if __name__ == '__main__':
    ast, errors, context, scope = run_pipeline(G, text)
    assert set(errors) == {
	 'Operation is not defined between "int" and "B".',
	 'Cannot convert "int" into "A".'
    }

=================== TEXT ======================

class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b + new B ( ) ;
    }
    b : int ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        d ;        
    }
}

================== TOKENS =====================
class id {
    id : id ;
    def id ( id : id , id : id ) : id {
        id + id + new id ( ) ;
    }
    id : id ;
}
class id : id {
    id : id ;
    def id ( id : id , id : id ) : id {
        let id : id = int ;
        let id = new id ( ) . id ( int , id ) ;
        id ;
    }
}
$
=================== PARSE =====================
<def-attr> -> id : id ;
<param> -> id : id
<param> -> id : id
<param-list> -> <param>
<param-list> -> <param> , <param-list>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <term>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <arith> + <term>
<atom> -> ne

In [19]:
text = '''
class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b + new B ( ) ;
    }
    b : int ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        d ;        
    }
    
    def test ( a : A , b : B ) : int {
        let self = 1 ;
        let z : int = a + b ;
        let z = a . suma ( 1 , 2 , 3 ) ;
        let w = b . suma ( a , b ) ;
        let w : int = b . resta ( 69 ) ;
        b ;
    }
}
'''

if __name__ == '__main__':
    ast, errors, context, scope = run_pipeline(G, text)

=================== TEXT ======================

class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b + new B ( ) ;
    }
    b : int ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        d ;        
    }
    
    def test ( a : A , b : B ) : int {
        let self = 1 ;
        let z : int = a + b ;
        let z = a . suma ( 1 , 2 , 3 ) ;
        let w = b . suma ( a , b ) ;
        let w : int = b . resta ( 69 ) ;
        b ;
    }
}

================== TOKENS =====================
class id {
    id : id ;
    def id ( id : id , id : id ) : id {
        id + id + new id ( ) ;
    }
    id : id ;
}
class id : id {
    id : id ;
    def id ( id : id , id : id ) : id {
        let id : id = int ;
        let id = new id ( ) . id ( int , id ) ;
        id ;
    }
    def id ( id : id , id : id ) : id {
        let id = int ;
        let id : id = id + id ;
        

## Propuestas

- Añada support (en una rama alternativa) para detectar fila y columna donde ocurrió el error.